In [1]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

/Users/lta/anaconda3/envs/llm_universe_2.x/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()
# 第13页为南瓜书第一页正文，因此从13页开始,从倒数13页涉及敏感用语，因此从-13页结束
data_pages = pdf_pages[13:-13]

In [3]:
import re

def clean_text(text: str):
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)

    return text

for page in data_pages:
    page.page_content = clean_text(page.page_content)

In [4]:
from generate_qa_pairs import QaPairs
qa_pairs = QaPairs.from_json('train_dataset.json')

In [5]:
from tqdm import tqdm

i = 0
for qa_pair in tqdm(qa_pairs.qa_pairs):
    if len(qa_pair['query']) > 10:
        i += 1
print('问答对成功生成率：' + str(i/len(qa_pairs.qa_pairs)))

100%|██████████| 119/119 [00:00<00:00, 1919700.68it/s]

问答对成功生成率：0.9243697478991597


In [6]:
from langchain.vectorstores.chroma import Chroma
def calculat_recall(k: int, vectordb: Chroma):
    i = 0
    j = 0
    for qa_pair in tqdm(qa_pairs.qa_pairs):
        if len(qa_pair['query']) > 10:
            query = qa_pair['query']
            sim_docs = vectordb.similarity_search(query,k=k)
            page_nums = [doc.metadata['page'] for doc in sim_docs]
            if qa_pair['page_num'] in page_nums: i += 1
            j += 1
    return i/j

In [7]:
from langchain_text_splitters.character import CharacterTextSplitter

chunk_mode_recall = []
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50, separator='')
split_docs = text_splitter.split_documents(data_pages)
vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding,
    )
chunk_mode_recall.append([calculat_recall(i, vectordb) for i in [1, 3, 5, 10]])

100%|██████████| 119/119 [00:00<00:00, 138.58it/s]


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, chunk_overlap=50, separators=['。', '，', ''])
split_docs = text_splitter.split_documents(data_pages)
vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding,
    )
chunk_mode_recall.append([calculat_recall(i, vectordb) for i in [1, 3, 5, 10]])

100%|██████████| 119/119 [00:00<00:00, 138.95it/s]


In [9]:
from langchain_experimental.text_splitter import SemanticChunker

sem_splitter = SemanticChunker(embeddings=embedding)
for page in data_pages: page = page.page_content.replace('。', '.').replace('？', '?').replace('！', '!')

split_docs = sem_splitter.split_documents(data_pages)
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, chunk_overlap=50, separators=['.', ',', ''])
split_docs = text_splitter.split_documents(split_docs)

vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding,
    )
chunk_mode_recall.append([calculat_recall(i, vectordb) for i in [1, 3, 5, 10]])

100%|██████████| 119/119 [00:00<00:00, 137.44it/s]


In [10]:
import csv
column_names = ['', 'top_1', 'top_3', 'top_5', 'top_10']
row_names = ['fixed_chunk', 'recursive_chunk', 'semantic_recursive_chunk']

with open('chunk_mode_recall.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(column_names)
    
    for i, row in enumerate(chunk_mode_recall):
        writer.writerow([row_names[i]] + row)